# Heart Disease Prediction Guided Project

This is a guided project from Dataquest's course "Introduction to Supervised Machine Learning in Python".

According to the World Health Organisation (WHO), an estimated 17.9 million people die each from cardiovascular diseases. There are manifold risk factors that potentially contribute to cardiovascular diseases in a person, such as an unhealthy diet, lack of physical activity, and mental illness.